In [30]:
import pandas as pd
import numpy as np

import sklearn.linear_model.logistic as logistic
from sklearn.ensemble import AdaBoostClassifier
from stockstats import StockDataFrame as Sdf


In [31]:
GASX = pd.read_csv("GASX.csv")
GASX.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,12/3/15,152.720001,163.199997,150.000000,40.799999,163.199997,200
1,12/4/15,179.679993,179.679993,179.520004,44.880001,179.520004,200
2,12/7/15,220.759995,239.259995,220.759995,59.509998,238.039993,1200
3,12/8/15,260.000000,269.679993,227.039993,57.537998,230.151993,3000
4,12/9/15,218.880005,236.679993,203.000000,56.740002,226.960007,900


In [32]:
GASX['percent_return'] = GASX['Adj Close'].pct_change()
GASX['Close'] = GASX['Adj Close']

In [33]:
GASX.head()

,Date,Open,High,Low,Close,Adj Close,Volume,percent_return
0,12/3/15,152.720001,163.199997,150.000000,163.199997,163.199997,200,NaN
1,12/4/15,179.679993,179.679993,179.520004,179.520004,179.520004,200,0.100000
2,12/7/15,220.759995,239.259995,220.759995,238.039993,238.039993,1200,0.325980
3,12/8/15,260.000000,269.679993,227.039993,230.151993,230.151993,3000,-0.033137
4,12/9/15,218.880005,236.679993,203.000000,226.960007,226.960007,900,-0.013869


In [34]:
stock_df = Sdf.retype(GASX)
GASX['rsi']=stock_df['rsi_10']
GASX['MACD_']=stock_df['macd']
#GASX['EMA']=stock_df['ema']
GASX.head()

/Users/yuanzhuang/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,open,high,low,close,adj close,volume,percent_return,close_-1_s,close_-1_d,rs_10,rsi_10,rsi,close_12_ema,close_26_ema,macd,macds,macdh,MACD_
date,,,,,,,,,,,,,,,,,,
12/3/15,152.720001,163.199997,150.000000,163.199997,163.199997,200,NaN,NaN,NaN,NaN,NaN,NaN,163.199997,163.199997,0.000000,0.000000,0.000000,0.000000
12/4/15,179.679993,179.679993,179.520004,179.520004,179.520004,200,0.100000,163.199997,16.320007,inf,100.000000,100.000000,172.040001,171.673847,0.366154,0.203419,0.325470,0.366154
12/7/15,220.759995,239.259995,220.759995,238.039993,238.039993,1200,0.325980,179.520004,58.519989,inf,100.000000,100.000000,197.799813,195.518559,2.281254,1.054991,2.452527,2.281254
12/8/15,260.000000,269.679993,227.039993,230.151993,230.151993,3000,-0.033137,238.039993,-7.888000,8.352839,89.308060,89.308060,208.012132,205.200554,2.811578,1.650041,2.323075,2.811578
12/9/15,218.880005,236.679993,203.000000,226.960007,226.960007,900,-0.013869,230.151993,-3.191986,5.762064,85.211617,85.211617,213.160202,210.246661,2.913541,2.025904,1.775275,2.913541


In [35]:
del GASX['close_-1_s']
del GASX['close_-1_d']
del GASX['rs_10']
del GASX['rsi_10']

del GASX['macd']
del GASX['macds']
del GASX['macdh']
GASX.head()

,open,high,low,close,adj close,volume,percent_return,rsi,close_12_ema,close_26_ema,MACD_
date,,,,,,,,,,,
12/3/15,152.720001,163.199997,150.000000,163.199997,163.199997,200,NaN,NaN,163.199997,163.199997,0.000000
12/4/15,179.679993,179.679993,179.520004,179.520004,179.520004,200,0.100000,100.000000,172.040001,171.673847,0.366154
12/7/15,220.759995,239.259995,220.759995,238.039993,238.039993,1200,0.325980,100.000000,197.799813,195.518559,2.281254
12/8/15,260.000000,269.679993,227.039993,230.151993,230.151993,3000,-0.033137,89.308060,208.012132,205.200554,2.811578
12/9/15,218.880005,236.679993,203.000000,226.960007,226.960007,900,-0.013869,85.211617,213.160202,210.246661,2.913541


In [36]:
def trinary_result(percent, threshold = 0.005):
    if (percent > threshold):
        return 1
    elif (percent < -threshold):
        return -1
    else:
        return 0

In [37]:
GASX['result_1'] = np.sign(GASX['percent_return'] )
GASX['result_1']= GASX.result_1.shift(-1)
GASX.head()

,open,high,low,close,adj close,volume,percent_return,rsi,close_12_ema,close_26_ema,MACD_,result_1
date,,,,,,,,,,,,
12/3/15,152.720001,163.199997,150.000000,163.199997,163.199997,200,NaN,NaN,163.199997,163.199997,0.000000,1.0
12/4/15,179.679993,179.679993,179.520004,179.520004,179.520004,200,0.100000,100.000000,172.040001,171.673847,0.366154,1.0
12/7/15,220.759995,239.259995,220.759995,238.039993,238.039993,1200,0.325980,100.000000,197.799813,195.518559,2.281254,-1.0
12/8/15,260.000000,269.679993,227.039993,230.151993,230.151993,3000,-0.033137,89.308060,208.012132,205.200554,2.811578,-1.0
12/9/15,218.880005,236.679993,203.000000,226.960007,226.960007,900,-0.013869,85.211617,213.160202,210.246661,2.913541,-1.0


In [41]:
GASX.tail()

,open,high,low,close,adj close,volume,percent_return,rsi,close_12_ema,close_26_ema,MACD_,result_1
date,,,,,,,,,,,,
6/23/17,44.987999,44.987999,42.110001,42.220001,42.220001,6100,-0.086543,56.457168,41.570639,38.516719,3.053920,-1.0
6/26/17,41.389999,42.182999,39.900002,40.972000,40.972000,7200,-0.029559,52.779208,41.478540,38.698592,2.779949,1.0
6/27/17,40.060001,41.500000,39.790001,41.410000,41.410000,7900,0.010690,53.949092,41.467996,38.899437,2.568559,-1.0
6/28/17,41.410000,41.410000,39.020000,39.020000,39.020000,3300,-0.057716,46.903803,41.091381,38.908367,2.183014,-1.0
6/29/17,39.242001,39.242001,36.779999,38.049999,38.049999,19000,-0.024859,44.295228,40.623476,38.844784,1.778692,NaN


In [44]:
features = ['open','high','low','close','volume','rsi','MACD_']


In [45]:
GASX_data = GASX[features][5:-1]
outcome = GASX['result_1'][5:-1]

In [46]:
logistic1 = logistic.LogisticRegression()
logistic1.fit(GASX_data[0:100],outcome[0:100])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [47]:
predictResult = logistic1.predict(GASX_data[100:-1])
isCorrect = predictResult == outcome[100:-1]
print(sum(isCorrect) / len(isCorrect))

0.487889273356


In [49]:
adaBoost = AdaBoostClassifier()
adaBoost.fit(GASX_data[0:100],outcome[0:100])

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)

In [50]:
predictResult2 = adaBoost.predict(GASX_data[100:-1])
isCorrect2 = predictResult2 == outcome[100:-1]
print(sum(isCorrect2) / len(isCorrect2))

0.467128027682
